# 准备测试集

In [ ]:
import pandas as pd
import numpy as np
import torch

# === 1. 读取 CSV ===
df = pd.read_csv('./data/Normal.csv', header=None)

# 手动指定列名（根据表2.1 + 空列处理）
columns = [
    'time', None, None, None, 'desired_pos',
    None, 'act_x_pos', 'motor_x_curr', 'motor_y_curr', 'motor_z_curr',
    'motor_x_volt', 'motor_y_volt', None,
    'motor_x_temp', 'motor_y_temp', 'motor_z_temp',
    'nut_x_temp', 'nut_y_temp', None,
    'ambient_temp'
]

# 丢弃空列（None 对应的列）
df.columns = columns
df = df.drop(columns=[col for col in df.columns if col is None])

# 转换时间列（可选）
df['time'] = pd.to_datetime(df['time'])

# === 2. 提取目标变量：Motor Y Voltage ===
target = df['motor_y_volt'].values  # shape: (N,)

# 如果做单特征预测，输入 = 输出 = motor_y_volt
data = target.reshape(-1, 1)  # shape: (N, 1)

# 如果做多特征预测，可拼接其他列，例如：
# features = ['motor_y_volt', 'motor_y_curr', 'motor_y_temp', 'ambient_temp']
# data = df[features].values  # shape: (N, 4)

print(f"原始数据形状: {data.shape}")  # e.g., (2000, 1)

## 构建滑动窗口测试样本

In [ ]:
def create_informer_testset(data, seq_len=200, label_len=30, pred_len=30):
    """
    构建 Informer 测试集
    返回: 
        X: [num_samples, seq_len, feature_dim]
        Y: [num_samples, pred_len, feature_dim]  # 真实未来值
    """
    X, Y = [], []
    total_len = len(data)
    
    # 滑动窗口步长 = pred_len（避免重叠预测）
    step = pred_len
    
    for i in range(0, total_len - seq_len - pred_len + 1, step):
        # 历史输入
        x = data[i : i + seq_len]
        # 未来真实值（用于评估）
        y = data[i + seq_len : i + seq_len + pred_len]
        
        X.append(x)
        Y.append(y)
    
    return np.array(X), np.array(Y)

# 构建测试集
seq_len = 200
label_len = 30
pred_len = 30

X_test, Y_test = create_informer_testset(data, seq_len, label_len, pred_len)

print(f"X_test shape: {X_test.shape}")  # e.g., (60, 200, 1)
print(f"Y_test shape: {Y_test.shape}")  # e.g., (60, 30, 1)

## 转换为 PyTorch Tensor（供模型使用）

In [ ]:
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
Y_test_tensor = torch.tensor(Y_test, dtype=torch.float32)

# 如果使用 GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X_test_tensor = X_test_tensor.to(device)
Y_test_tensor = Y_test_tensor.to(device)

In [ ]:
import torch
from models.informer import Informer
from data_loader import DataLoader  # 或你自己写的 loader

# 初始化模型
model = Informer(
    seq_len=200,
    label_len=30,
    pred_len=30,
    enc_in=1,  # 输入特征维度
    dec_in=1,
    c_out=1,   # 输出维度
)

# 加载训练好的权重
model.load_state_dict(torch.load('checkpoints/informer_best.pth'))
model.eval()

# 推理
with torch.no_grad():
    pred = model(X_test)  # shape: [B, 30, 1]
    pred = pred.cpu().numpy()
    true = y_test.cpu().numpy()